In [0]:
import json
import numpy as np
import pandas as pd
from datetime import *
from pyspark import SparkContext
from pyspark.sql import SparkSession, SQLContext
from pyspark.sql.functions import *
from pyspark.sql.types import * 
from pyspark.sql.types import BooleanType, StructType, StructField, StringType, IntegerType, MapType, ArrayType
from pyspark.sql.functions import from_json
from pyspark.sql import DataFrame

In [0]:
spark_conf={
        "spark.databricks.delta.properties.defaults.autoOptimize.optimizeWrite" : True,
        "spark.databricks.delta.properties.defaults.autoOptimize.autoCompact" :  True,
        "spark.databricks.delta.optimizeWrite.enabled": True,
        "spark.databricks.adaptive.autoOptimizeShuffle.enabled": True,
        "spark.databricks.adaptive.skewJoin.spillProof.enabled": True,
        "spark.sql.adaptive.skewJoin.enabled": True,
        "spark.databricks.delta.cache.enabled": True,
        "spark.databricks.io.cache.enabled": True,
        "spark.sql.adaptive.coalescePartitions.enabled": True,
        "spark.sql.shuffle.partitions":"auto",
        "spark.databricks.optimizer.adaptive.enabled":True
}

In [0]:
inicio = pd.Timestamp("2023-10-01")

last_60d = date.today() - timedelta(days = 61)
print(last_60d)

first_day = date.today().replace(day=1)
# print(first_day)

last_13m = first_day - timedelta(days = 365)
print(last_13m)

### DLT METHODS

In [0]:
from dataclasses import asdict, dataclass, field
from pyspark.sql import DataFrame
from typing import Any, Callable, Dict, List, Union, Optional, Dict, Literal
import dlt

QUARANTINE_COL: str = "is_quarantined"

def can_quarantine(is_quarantined: bool, expect_all: Dict[str, str]) -> bool:
    quarantine: bool = is_quarantined and bool(expect_all) 
    #logger.debug(f"Can quarantine: {quarantine}")
    return quarantine


def quarantine_rules(is_quarantined: bool, expect_all: Dict[str, str]) -> str:
    rules: str = (
        "NOT({0})".format(" AND ".join(expect_all.values()))
        if can_quarantine(is_quarantined, expect_all)
        else "1=0"
    )
    #logger.debug(f"Quarantine rules: {rules}")
    return rules

In [0]:
def generate_view(input_df: DataFrame, destination: str, comment: str, expect_all: Dict[str, str] = {}, expect_all_or_drop: Dict[str, str] = {}, expect_all_or_fail: Dict[str, str] ={}, spark_conf: Dict[str,str] = {}) -> None:
  """
  Generates a view for the specified dataframe.
  """
  #logger.info(f"Generating view for entity: ")
  @dlt.view(
      name=destination,
      comment=comment,
      spark_conf=spark_conf
  )
  @dlt.expect_all(expectations=expect_all)
  @dlt.expect_all_or_drop(expectations=expect_all_or_drop)
  @dlt.expect_all_or_fail(expectations=expect_all_or_fail)
  def _():
    return input_df

In [0]:
def generate_table(input_df: DataFrame, destination: str, partition_cols: List[str], comment: str, expect_all: Dict[str, str] = {}, expect_all_or_drop: Dict[str, str] = {}, expect_all_or_fail: Dict[str, str] = {}, is_quarantined: bool = False, has_scd: bool = False, src: str = '', keys: List[str] =[], sequence_by: str = '', ignore_null_updates: bool = False, apply_as_deletes: str = '', spark_conf: Dict[str,str] = {}) -> None:
    """
    Generates a table for the specified entity.
    """
    #logger.info(f"Generating table for entity: ")
    name: str = destination
    quarantine_name: str = f"{name}_quarantine"
    invalid_name: str = f"{name}_invalid"
    if can_quarantine(is_quarantined, expect_all):
        _create_quarantine_tables(
            valid_name=name,
            invalid_name=invalid_name,
            quarantine_name=quarantine_name,
            partition_cols = partition_cols)
        name = quarantine_name
        partition_cols = [QUARANTINE_COL] + partition_cols
    if has_scd:
        _create_scd_table(source=src, keys=keys, sequence_by=sequence_by, ignore_null_updates=ignore_null_updates, apply_as_deletes=apply_as_deletes, name=name, partition_cols=partition_cols, comment=comment, expect_all=expect_all, expect_all_or_drop=expect_all_or_drop, expect_all_or_fail=expect_all_or_fail, spark_conf = spark_conf)
    else:
        _create_table(input_df, name, partition_cols, comment, expect_all, expect_all_or_drop, expect_all_or_fail, is_quarantined, spark_conf = spark_conf)

In [0]:
def _create_quarantine_tables(
    valid_name: str,
    invalid_name: str,
    quarantine_name: str,
    partition_cols: List[str],
):
  @dlt.table(name=valid_name, partition_cols=partition_cols)
  def valid_data():
    df: DataFrame = (
        dlt.readStream(quarantine_name)
    )
    return df.filter(f"{QUARANTINE_COL}=false").drop(QUARANTINE_COL, "_rescued_data")

  @dlt.table(name=invalid_name, partition_cols=partition_cols)
  def invalid_data():
    df: DataFrame = (
        dlt.readStream(quarantine_name)
    )
    return df.filter(f"{QUARANTINE_COL}=true").drop(QUARANTINE_COL)

In [0]:
def _create_table(input_df: DataFrame, name: str, partition_cols: List[str], comment: str, expect_all: Dict, expect_all_or_drop: Dict, expect_all_or_fail: Dict, is_quarantined: bool, spark_conf: Dict):
  #logger.debug(f"Creating table: {name}")
  is_temporary: bool = is_quarantined
  @dlt.table(
    name=name,
    comment=comment,
    partition_cols=partition_cols,
    spark_conf=spark_conf,
    temporary=is_temporary,
  )
  @dlt.expect_all(expectations=expect_all)
  @dlt.expect_all_or_drop(expectations=expect_all_or_drop)
  @dlt.expect_all_or_fail(expectations=expect_all_or_fail)
  def target_table():
    df: DataFrame = input_df
    if can_quarantine(is_quarantined, expect_all):
      rules: str = quarantine_rules(is_quarantined, expect_all)
      df = df.withColumn(QUARANTINE_COL, F.expr(rules))
    return df

In [0]:
def _create_scd_table(source:str, keys: List[str], sequence_by: str, ignore_null_updates: bool, apply_as_deletes: str, name: str, partition_cols: List[str], comment: str, expect_all: Dict, expect_all_or_drop: Dict, expect_all_or_fail: Dict, spark_conf: Dict
):
  #logger.debug(f"Creating SCD table: {name}")
  dlt.create_streaming_table(
      name=name,
      comment=comment,
      partition_cols=partition_cols,
      spark_conf=spark_conf,
      expect_all=expect_all,
      expect_all_or_drop=expect_all_or_drop,
      expect_all_or_fail=expect_all_or_fail,
  )
  dlt.apply_changes(
      target=name,
      source=source,
      keys=keys,
      sequence_by=sequence_by,
      ignore_null_updates=ignore_null_updates,
      apply_as_deletes=apply_as_deletes
  )

In [0]:
a = f"""
SELECT 
    cd_simulation,
    consumer_cpf,
    CAST(dt_created_at AS DATE)              AS dt_simulation,
    date_format(dt_created_at, 'yyyyMM')     AS anomes,
    consumerNegativated,
    ds_client,
    vl_simulation,

    CASE
    WHEN ds_client = "ecred-serasa-web'" THEN 'ecred-serasa-web' 
    ELSE ds_client 
    END AS canal,
     
    offerHallGatherKey,

    CASE WHEN reason_key = 'credit-card' THEN 'Cartão'
    ELSE 'Empréstimo'
    END AS ds_product1,

    CASE 
    WHEN cardinality(ls_products_events) >= 1 THEN 1 
    ELSE                                    0 
    END AS fl_elegivel_oferta,

    CASE 
    WHEN cardinality(ls_products_events) >= 1 THEN 1 
    ELSE                                    0 
    END AS fl_chamado,
	
    CASE 
    WHEN cardinality(ls_offers_events) >= 1 THEN 1 
    ELSE                                  0 
    END AS fl_offer
    
FROM db_ecred_silver.tb_new_events_simulation
WHERE dt_event >= '{inicio}'
      AND ds_client in ('ecred-web', 'ecred-app', 'app-ios', 'app-android', 'serasa-web')
"""

simulacao = spark.sql(a)
generate_view(input_df=simulacao, comment= 'SOME RELEVANT COMMENT HERE', spark_conf=spark_conf, destination='temp_simulacao')

#simulacao.createOrReplaceTempView('temp_simulacao')
# simulacao.count()

In [0]:
df_eventos = spark.sql(f'''
  SELECT *
  FROM db_ecred_silver.tb_events_integration
  WHERE dt_event >= '{inicio}'
  AND cd_event IN (2600,2602,311)
''')

# df_eventos.createOrReplaceTempView('temp_df_eventos')

In [0]:
schema_event_2600 = StructType([ \
    StructField("consumerScoresHSPN",StringType(),True), \
    StructField("consumerProfessionLabel",StringType(),True), \
    StructField("consumerOccupationLabel",StringType(),True), \
    StructField("consumerAge",StringType(),True), \
    StructField("consumerIncome",StringType(),True), \
    StructField("consumerScoresHRP9",StringType(),True), \
  ])

In [0]:
df_event_2600 = df_eventos.filter(df_eventos.cd_event == 2600).withColumn("obj_ds_data", from_json(df_eventos.ds_data, schema_event_2600))

In [0]:
df_event_2600 = df_event_2600.withColumn("consumerScoresHSPN", df_event_2600.obj_ds_data.consumerScoresHSPN)\
                             .withColumn("consumerProfessionLabel", df_event_2600.obj_ds_data.consumerProfessionLabel)\
                             .withColumn("consumerOccupationLabel", df_event_2600.obj_ds_data.consumerOccupationLabel)\
                             .withColumn("consumerAge", df_event_2600.obj_ds_data.consumerAge)\
                             .withColumn("consumerIncome", df_event_2600.obj_ds_data.consumerIncome)\
                             .withColumn("consumerScoresHRP9", df_event_2600.obj_ds_data.consumerScoresHRP9)

In [0]:
generate_view(input_df=df_event_2600, comment= 'SOME RELEVANT COMMENT HERE', spark_conf=spark_conf, destination='temp_df_event_2600')

In [0]:
a = f"""
SELECT
    a.*,
    CAST(b.consumerScoresHSPN AS int)      AS consumerScoresHSPN,
    CAST(b.consumerAge AS int)             AS consumerAge,
    CAST(b.consumerIncome AS int)          AS consumerIncome,
    CAST(b.consumerScoresHRP9 AS int)      AS consumerScoresHRP9,
    b.consumerProfessionLabel,
    b.consumerOccupationLabel
    -- ,CASE WHEN b.cd_simulation IS NULL THEN 0 ELSE 1 END AS fl_join_2600
    
FROM LIVE.temp_simulacao AS a
  LEFT JOIN LIVE.temp_df_event_2600 AS b
    ON a.cd_simulation = b.cd_simulation
"""

simulacao2 = spark.sql(a)

generate_view(input_df=simulacao2, comment= 'SOME RELEVANT COMMENT HERE', spark_conf=spark_conf, destination='temp_simulacao2')
#simulacao2.createOrReplaceTempView('temp_simulacao2')
# simulacao2.count()

In [0]:
schema_event_2602 = StructType([ \
    StructField("occupationLabel",StringType(),True), \
    StructField("professionLabel",StringType(),True), \
    StructField("addressStateValue",StringType(),True), \
    StructField("addressCity",StringType(),True), \
  ])

In [0]:
df_event_2602 = df_eventos.filter(df_eventos.cd_event == 2602).withColumn("obj_ds_data", from_json(df_eventos.ds_data, schema_event_2602))

In [0]:
df_event_2602 = df_event_2602.withColumn("occupationLabel", df_event_2602.obj_ds_data.occupationLabel)\
                             .withColumn("professionLabel", df_event_2602.obj_ds_data.professionLabel)\
                             .withColumn("addressStateValue", df_event_2602.obj_ds_data.addressStateValue)\
                             .withColumn("addressCity", df_event_2602.obj_ds_data.addressCity)

In [0]:
generate_view(input_df=df_event_2602, comment= 'SOME RELEVANT COMMENT HERE', spark_conf=spark_conf, destination='temp_df_event_2602')

In [0]:
a = f"""
SELECT
    a.*,
    b.addressStateValue
    -- ,CASE WHEN b.cd_simulation IS NULL THEN 0 ELSE 1 END AS fl_join_2602
    
FROM LIVE.temp_simulacao2 AS a
  LEFT JOIN LIVE.temp_df_event_2602 AS b
    ON a.cd_simulation = b.cd_simulation
"""

simulacao3 = spark.sql(a)
generate_view(input_df=simulacao3, comment= 'SOME RELEVANT COMMENT HERE', spark_conf=spark_conf, destination='temp_simulacao3')
#simulacao3.createOrReplaceTempView('temp_simulacao3')
# simulacao3.count()

In [0]:
schema_event_311 = StructType([ \
    StructField("creditTypeKeyV2",StringType(),True), \
    StructField("productId",StringType(),True), \
  ])

In [0]:
df_event_311 = df_eventos.filter(df_eventos.cd_event == 311).withColumn("obj_ds_data", from_json(df_eventos.ds_data, schema_event_311))

In [0]:
df_event_311 = df_event_311.withColumn("creditTypeKeyV2", df_event_311.obj_ds_data.creditTypeKeyV2)\
                         .withColumn("productId", df_event_311.obj_ds_data.productId)

In [0]:
generate_view(input_df=df_event_311, comment= 'SOME RELEVANT COMMENT HERE', spark_conf=spark_conf, destination='temp_df_event_311')

In [0]:
a = f"""
select
cd_simulation,

MAX(CASE WHEN creditTypeKeyV2 in ('personal-loan', 'personal-loan-fgts', 'vehicle-guarantee-loan', 'home-equity', 'payroll-loan') THEN 1 ELSE 0 END) AS fl_ep_311,
MAX(CASE WHEN creditTypeKeyV2 in ('credit-card')                                                                                  THEN 1 ELSE 0 END) AS fl_cartao_311,
MAX(CASE WHEN creditTypeKeyV2 in ('digital-account')                                                                              THEN 1 ELSE 0 END) AS fl_conta_311,

MAX(CASE WHEN productId in ('118', '131', '136', '149', '161', '166', '169', '185', '186', '197', '199', '76')           THEN 1 ELSE 0 END) AS fl_cartao_gap,
MAX(CASE WHEN productId in ('177')                                                                                       THEN 1 ELSE 0 END) AS fl_conta_gap,
MAX(CASE WHEN productId in ('14', '145', '160', '171', '196', '29', '4', '162', '178', '189', '9', '188', '137', '198')  THEN 1 ELSE 0 END) AS fl_ep_gap

FROM LIVE.temp_df_event_311
GROUP BY 1
"""

aux_df_event_311 = spark.sql(a)
generate_view(input_df=aux_df_event_311, comment= 'SOME RELEVANT COMMENT HERE', spark_conf=spark_conf, destination='temp_aux_df_event_311')


In [0]:
a = f"""
SELECT
    a.*,
    b.fl_ep_311,
    b.fl_cartao_311,
    b.fl_conta_311,
    b.fl_cartao_gap,
    b.fl_conta_gap,
    b.fl_ep_gap
    -- ,CASE WHEN b.cd_simulation IS NULL THEN 0 ELSE 1 END AS fl_join_311
    
FROM LIVE.temp_simulacao3 AS a
  LEFT JOIN LIVE.temp_aux_df_event_311 AS b
    ON a.cd_simulation = b.cd_simulation
"""

simulacao4 = spark.sql(a)
generate_view(input_df=simulacao4, comment= 'SOME RELEVANT COMMENT HERE', spark_conf=spark_conf, destination='temp_simulacao4')
#simulacao4.createOrReplaceTempView('temp_simulacao4')
# simulacao4.count()

In [0]:
a = f"""
SELECT
*,   

CASE WHEN ds_product1 = 'Cartão'     THEN 1 ELSE 0 END AS fl_cartao_reason,
CASE WHEN ds_product1 = 'Empréstimo' THEN 1 ELSE 0 END AS fl_ep_reason,
0 AS fl_conta_reason,

CASE WHEN consumerScoresHSPN > 900                              THEN '01. 900+'
     WHEN consumerScoresHSPN > 800 and consumerScoresHSPN <=900 THEN '02. 801-900'
     WHEN consumerScoresHSPN > 700 and consumerScoresHSPN <=800 THEN '03. 701-800'
     WHEN consumerScoresHSPN > 600 and consumerScoresHSPN <=700 THEN '04. 601-700'
     WHEN consumerScoresHSPN > 500 and consumerScoresHSPN <=600 THEN '05. 501-600'
     WHEN consumerScoresHSPN > 400 and consumerScoresHSPN <=500 THEN '06. 401-500'
     WHEN consumerScoresHSPN > 300 and consumerScoresHSPN <=400 THEN '07. 301-400'
     WHEN consumerScoresHSPN > 200 and consumerScoresHSPN <=300 THEN '08. 201-300'
     WHEN consumerScoresHSPN > 100 and consumerScoresHSPN <=200 THEN '09. 101-200'
     WHEN consumerScoresHSPN > 0   and consumerScoresHSPN <=100 THEN '10. 0-100'
     ELSE null
END AS fx_consumerScoresHSPN,

CASE WHEN consumernegativated = 'true' THEN 'Negativado'
     WHEN consumerScoresHSPN < 700    THEN '0 a 699'
     WHEN consumerScoresHSPN >= 700   THEN '700+'
END AS qualidade,

CASE WHEN consumerScoresHSPN >= 0  and consumerScoresHSPN <= 300 THEN 'G04 - Negativado e baixíssima renda'
     WHEN consumerScoresHSPN > 300 and consumerScoresHSPN <= 600 THEN 'G03 - Negativado com baixa renda mas com capacidade de pagamento'
     WHEN consumerScoresHSPN > 600 and consumerScoresHSPN <= 800 THEN 'G02 - Não Negativado com renda média e alta capacidade de pontualidade em pagamentos'
     WHEN consumerScoresHSPN > 800                               THEN 'G01 - Não Negativado com maior renda e alta capacidade de pontualidade em pagamentos'
ELSE 'NA'
END AS cluster_hspn_puro,

CASE WHEN consumerScoresHRP9 >= 20000                                THEN  '07. 20k+'
     WHEN consumerScoresHRP9 >= 10000 and consumerScoresHRP9 < 20000 THEN  '06. 10k-20k'
     WHEN consumerScoresHRP9 >= 8000  and consumerScoresHRP9 < 10000 THEN  '05. 8k-10k'
     WHEN consumerScoresHRP9 >= 6000  and consumerScoresHRP9 < 8000  THEN  '04. 6k-8k'
     WHEN consumerScoresHRP9 >= 4000  and consumerScoresHRP9 < 6000  THEN  '03. 4k-6k'
     WHEN consumerScoresHRP9 >= 2000  and consumerScoresHRP9 < 4000  THEN  '02. 2k-4k'
     WHEN consumerScoresHRP9 >= 0     and consumerScoresHRP9 < 2000  THEN  '01. 0-2k'   
     ELSE null
END AS fx_consumerScoresHRP9,

CASE WHEN consumerIncome >= 20000                            THEN  '07. 20k+'
     WHEN consumerIncome >= 10000 and consumerIncome < 20000 THEN  '06. 10k-20k'
     WHEN consumerIncome >= 8000  and consumerIncome < 10000 THEN  '05. 8k-10k'
     WHEN consumerIncome >= 6000  and consumerIncome < 8000  THEN  '04. 6k-8k'
     WHEN consumerIncome >= 4000  and consumerIncome < 6000  THEN  '03. 4k-6k'
     WHEN consumerIncome >= 2000  and consumerIncome < 4000  THEN  '02. 2k-4k'
     WHEN consumerIncome >= 0     and consumerIncome < 2000  THEN  '01. 0-2k'   
     ELSE null
END AS fx_consumerIncome,

CASE WHEN consumerAge >= 60                      THEN '06. 60+'
     WHEN consumerAge >= 50 and consumerAge < 60 THEN '05. 50-59'
     WHEN consumerAge >= 40 and consumerAge < 50 THEN '04. 40-49'
     WHEN consumerAge >= 30 and consumerAge < 40 THEN '03. 30-39'
     WHEN consumerAge >= 20 and consumerAge < 30 THEN '02. 20-29'
     WHEN consumerAge >= 0 and consumerAge < 20  THEN '01. 0-19'
     ELSE null
END AS fx_consumerAge,

CASE WHEN addressStateValue IS NULL                                             THEN 'Null'
     WHEN addressStateValue IN ("DF","GO","MS","MT") 						THEN 'CO'
     WHEN addressStateValue IN ("AC","AM","AP","PA","RO","RR","TO") 	          THEN 'N'
     WHEN addressStateValue IN ("AL","BA","CE","MA","PB","PE","PI","RN","SE")   THEN 'NE'
     WHEN addressStateValue IN ("ES","MG","RJ","SP") 						THEN 'SE'
     WHEN addressStateValue IN ("PR","RS","SC") 							THEN 'SUL'
     ELSE                                                                            'nda'
END AS regiao
    
FROM LIVE.temp_simulacao4
"""

simulacao5 = spark.sql(a)
generate_view(input_df=simulacao5, comment= 'SOME RELEVANT COMMENT HERE', spark_conf=spark_conf, destination='temp_df_nivel_simulacao5')
#simulacao5.createOrReplaceTempView('temp_df_nivel_simulacao5')
# simulacao5.count()

In [0]:
a = f"""
select
  cd_simulation,
  consumer_cpf,
  dt_simulation,
  anomes,
  consumerNegativated,
  ds_client,
  vl_simulation,
  canal,
  offerHallGatherKey,
  fl_elegivel_oferta,
  fl_chamado,
  fl_offer,
  consumerScoresHSPN,
  consumerAge,
  consumerIncome,
  consumerScoresHRP9,
  consumerProfessionLabel,
  consumerOccupationLabel,
  addressStateValue,
  fx_consumerScoresHSPN,
  qualidade,
  cluster_hspn_puro,
  fx_consumerScoresHRP9,
  fx_consumerIncome,
  fx_consumerAge,
  regiao,

  CASE WHEN dt_simulation < '2023-10-20'                       THEN fl_cartao_reason
      WHEN dt_simulation between '2023-10-20' and '2023-11-09' THEN fl_cartao_gap
      ELSE                                                          fl_cartao_311
  END AS fl_cartao,

  CASE WHEN dt_simulation < '2023-10-20'                       THEN fl_conta_reason
      WHEN dt_simulation between '2023-10-20' and '2023-11-09' THEN fl_conta_gap
      ELSE                                                          fl_conta_311
  END AS fl_conta,

  CASE WHEN dt_simulation < '2023-10-20'                       THEN fl_ep_reason
      WHEN dt_simulation between '2023-10-20' and '2023-11-09' THEN fl_ep_gap
      ELSE                                                          fl_ep_311
  END AS fl_ep

FROM LIVE.temp_df_nivel_simulacao5
"""

simulacao6 = spark.sql(a)
generate_view(input_df=simulacao6, comment= 'SOME RELEVANT COMMENT HERE', spark_conf=spark_conf, destination='temp_simulacao6')
#simulacao6.createOrReplaceTempView('temp_simulacao6')
# simulacao6.count()

In [0]:
a = f"""
SELECT DISTINCT
    cd_simulation_id,
    nu_document,
    cd_uuid,
    cd_order,
    ds_key,
    SUBSTR(dt_created, 1, 7)    AS anomes,
    CAST(dt_created as DATE)    AS dt_pedido,
    ds_client                   AS canal,
    vl_order,
    vl_contracted, 
    
    CASE
    WHEN ds_order_status_key NOT IN ('waiting-for-review',
                                     'cancelled-by-address-changed',
                                     'expired',
                                     'waiting-for-answers',
                                     'waiting-for-redirect',
                                     'waiting-for-phone-validation',
                                     'waiting-for-bank-validation',
                                     'expired-review',
                                     'expired-answers',
                                     'expired-bank-validation',
                                     'expired-phone-validation',
                                     'expired-review-offer',
                                     'expired-redirect',
                                     'invalid-bank-account') THEN 1 
    ELSE                                                          0 
    END AS flag_confirmado,
    CASE WHEN ds_order_status IN ('Aprovado', 'Liberado') THEN 1 
    ELSE                                                       0 
    END AS flag_aprovado,
    
    CASE 
    WHEN ds_credit_type LIKE 'Cart%' THEN 'Cartão'
    WHEN ds_credit_type LIKE 'Empr%' THEN 'Empréstimo'
    ELSE 								  'Outros' 
    END AS ds_product,
    
    cd_credit_type,
    ds_credit_type_key,
    ds_credit_type,
    ds_variant
    
FROM db_ecred_silver.tb_order_report
WHERE CAST(dt_created AS DATE) >= '{inicio}'
"""

df_pedidos = spark.sql(a)
generate_view(input_df=df_pedidos, comment= 'SOME RELEVANT COMMENT HERE', spark_conf=spark_conf, destination='temp_tbl_pedidos')
#df_pedidos.createOrReplaceTempView('temp_tbl_pedidos')
# df_pedidos.count()

In [0]:
a = f"""
SELECT DISTINCT
    a.*,
    b.ds_key,
    b.vl_order,

    CASE 
    WHEN b.flag_aprovado = 1 AND b.vl_contracted > 0  THEN b.vl_contracted 
    WHEN b.flag_aprovado = 1 AND b.vl_contracted <= 0 THEN b.vl_order
    ELSE                                                   null
    END AS vl_liberado,

    CASE WHEN b.cd_order        IS NOT NULL THEN 1 ELSE 0                 END AS flag_pedido,
    CASE WHEN b.flag_confirmado IS NULL     THEN 0 ELSE b.flag_confirmado END AS flag_confirmado,
    CASE WHEN b.flag_aprovado   IS NULL     THEN 0 ELSE b.flag_aprovado   END AS flag_aprovado
    
FROM LIVE.temp_simulacao6 AS a
    LEFT JOIN LIVE.temp_tbl_pedidos AS b
    ON a.cd_simulation = b.cd_simulation_id
    --AND a.partner_Key = b.ds_key
    --AND a.creditTypeKey = b.ds_credit_type_key
"""

simulacao7 = spark.sql(a)
generate_view(input_df=simulacao7, comment= 'SOME RELEVANT COMMENT HERE', spark_conf=spark_conf, destination='temp_simulacao7')
#simulacao7.createOrReplaceTempView('temp_simulacao7')
# simulacao7.count()

In [0]:
a = f"""
SELECT DISTINCT
anomes,
dt_simulation,
offerHallGatherKey,
fl_ep,
fl_cartao,
fl_conta,
ds_client,
-- partner_key,                                     (SÓ PARA FUNIL POR PARCEIRO)
-- description,                                     (SÓ PARA FUNIL POR SUBPRODUTO)
fx_consumerScoresHSPN,
qualidade,
fx_consumerScoresHRP9,
fx_consumerIncome,
fx_consumerAge,
regiao,
consumerOccupationLabel,
cluster_hspn_puro,
cd_simulation,
fl_elegivel_oferta,
fl_chamado,
fl_offer,
flag_pedido,
flag_confirmado,
flag_aprovado

from LIVE.temp_simulacao7

"""

df_analitico = spark.sql(a)
generate_view(input_df=df_analitico, comment= 'SOME RELEVANT COMMENT HERE', spark_conf=spark_conf, destination='temp_df_analitico')
#df_analitico.createOrReplaceTempView('temp_df_analitico')
# df_analitico.count()

### GERA ANALÍTICO PARA JOIN DO 1° CÓDIGO

In [0]:
generate_table(input_df=df_analitico, comment= 'SOME RELEVANT COMMENT HERE', destination='dash_vida_macroproduto_analitico_events2')

In [0]:
a = f"""
select * from db_ecred.dash_vida_macroproduto_analitico_events1
UNION all
select * from LIVE.dash_vida_macroproduto_analitico_events2
"""
df_analitico_events_final = spark.sql(a)
generate_table(input_df=df_analitico_events_final, comment= 'SOME RELEVANT COMMENT HERE', destination='dash_vida_macroproduto_analitico_events_final')

### MARCAÇÃO DE DATAS E GERAÇÃO DO CUBO FINAL 

In [0]:
a = f"""
SELECT
    *,
    CASE
    WHEN dt_simulation >= '{last_60d}' THEN 1
    ELSE                                    0
    END AS last_60d,

    CASE
    WHEN dt_simulation < '{last_60d}' THEN NULL
    ELSE                                   dt_simulation
    END AS dt_simulation_60d,

    CASE
    WHEN dt_simulation >= '{last_13m}' THEN 1
    ELSE                                    0
    END AS last_13m

FROM db_ecred.dash_vida_macroproduto_analitico_events_final
"""

df_funil_produto_datas = spark.sql(a)
generate_view(input_df=df_funil_produto_datas, comment= 'SOME RELEVANT COMMENT HERE', destination='temp_df_funil_produto_datas')


In [0]:
a= f"""
SELECT
anomes as anomes_simulacao,
dt_simulation_60d,
last_60d,
last_13m,
offerHallGatherKey,
fl_ep,
fl_cartao,
fl_conta,
ds_client,
fx_consumerScoresHSPN,
qualidade,
fx_consumerScoresHRP9,
fx_consumerIncome,
fx_consumerAge,
regiao,
consumerOccupationLabel,
cluster_hspn_puro,
COUNT(cd_simulation)        as qt_simulacao,
SUM(fl_elegivel_oferta)     as qt_elegivel,
SUM(fl_chamado)             as qt_chamado,
SUM(fl_offer)               as qt_oferta,
SUM(flag_pedido)            as qt_pedido,
SUM(flag_confirmado)        as qt_confirmado,
SUM(flag_aprovado)          as qt_aprovado

from LIVE.temp_df_funil_produto_datas
group by 1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17
"""
df_dash_vida_tableau_macroproduto = spark.sql(a)
generate_table(input_df=df_dash_vida_tableau_macroproduto, comment= 'SOME RELEVANT COMMENT HERE', destination='dash_vida_tableau_macroproduto')